_Lambda School Data Science, Unit 2_
 
# Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [1]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install packages in Colab
    !pip install category_encoders==2.0.0
    !pip install pandas-profiling==2.3.0
    !pip install plotly==4.1.1

In [2]:
# Read data
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)

# Check data shape
assert df.shape == (13958, 20)

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?

**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.

**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 

**7.** Get your model's **test accuracy.** (One time, at the end.)


**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**

### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.



## 1. Begin with baselines for classification. 

>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

In [4]:
# value_counts at [0] will return the most frequent value.
print(f'Baseline Accuracy: {df.shot_made_flag.value_counts()[0] / len(df.shot_made_flag)}')

Baseline Accuracy: 0.5270812437311936


## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [5]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0


In [6]:
type(df.game_date[0])

str

In [7]:
# no thank you. datetime please.
df['game_date'] = pd.to_datetime(df['game_date'])
type(df.game_date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
start = '10/1/2018'
train = df[df['game_date'] < start]
test = df[df['game_date'] > start]
print(train.shape)
print(test.shape)

(12249, 20)
(1709, 20)


## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

    

In [9]:
homecourt_advantage = []
for team in df.htm:
    if team == 'GSW':
        homecourt_advantage.append(1)
    else:
        homecourt_advantage.append(0)
homecourt_advantage[:20]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
df['homecourt_advantage'] = homecourt_advantage
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,...,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot,homecourt_advantage
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),...,26,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0,1
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),...,18,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0,1
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),...,14,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0,1
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),...,19,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0,1
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),...,16,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0,1


## **4. Decide how to validate** your model. 

>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [11]:
train, val = train_test_split(train, test_size=0.2, train_size=0.8, random_state=200)
print(train.shape)
print(val.shape)

(9799, 20)
(2450, 20)


## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [12]:
df.isna().sum()

game_id                    0
game_event_id              0
player_name                0
period                     0
minutes_remaining          0
seconds_remaining          0
action_type                0
shot_type                  0
shot_zone_basic            0
shot_zone_area             0
shot_zone_range            0
shot_distance              0
loc_x                      0
loc_y                      0
shot_made_flag             0
game_date                  0
htm                        0
vtm                        0
season_type                0
scoremargin_before_shot    0
homecourt_advantage        0
dtype: int64

In [13]:
# nothing needs to be imputed. moving on.
target = 'shot_made_flag'

# game date needs to be a string to work with the encoder
train['game_date'] = train['game_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
val['game_date'] = val['game_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
test['game_date'] = test['game_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test.drop(columns=target)
y_test = test[target]

C:\Users\redfo\Anaconda3\envs\unit2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [14]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestClassifier(random_state=200, n_jobs=-1)
)

# gonna put this through some cross validation to boost my accuracy a little
param_distributions = {
    'randomforestclassifier__n_estimators': randint(20, 250),
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1),
    'randomforestclassifier__min_samples_split': range(2, 10),
    'randomforestclassifier__min_samples_leaf': range(1, 10)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=15,
    cv=10,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
)

search.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 15 candidates, totalling 150 fits
[CV] randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.5076805274756948, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=104 
[CV]  randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.5076805274756948, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=104, score=(train=0.668, test=0.666), total=   1.7s
[CV] randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.5076805274756948, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=104 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.5076805274756948, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=104, score=(train=0.676, test=0.660), total=   0.5s
[CV] randomforestclassifier__max_depth=5, randomforestclassifier__max_features=0.5076805274756948, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=5, randomforestclassifier__n_estimators=104 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s


...

[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.1931044994859461, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=137, score=(train=0.968, test=0.663), total=   0.7s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.1931044994859461, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=137 
[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.1931044994859461, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=137, score=(train=0.965, test=0.637), total=   0.7s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.1931044994859461, randomforestclassifier__min_samples_leaf=1, randomforestclassifier__min_samples_split=8, randomforestclassifier

[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.47586052909118826, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=143, score=(train=0.970, test=0.628), total=   1.3s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.47586052909118826, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=143 
[CV]  randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.47586052909118826, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=8, randomforestclassifier__n_estimators=143, score=(train=0.965, test=0.659), total=   1.1s
[CV] randomforestclassifier__max_depth=20, randomforestclassifier__max_features=0.47586052909118826, randomforestclassifier__min_samples_leaf=2, randomforestclassifier__min_samples_split=8, randomforestclassi

[CV]  randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.4899829415320569, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=39, score=(train=0.805, test=0.671), total=   0.5s
[CV] randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.4899829415320569, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=39 
[CV]  randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.4899829415320569, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=2, randomforestclassifier__n_estimators=39, score=(train=0.802, test=0.647), total=   0.5s
[CV] randomforestclassifier__max_depth=15, randomforestclassifier__max_features=0.4899829415320569, randomforestclassifier__min_samples_leaf=9, randomforestclassifier__min_samples_split=2, randomforestclassifier__n

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  2.2min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(bootstrap=True,
                                                                     class_weight=None,
                              

## 6.Get your model's validation accuracy

> (Multiple times if you try multiple iterations.)

In [15]:
# let's make sure we're using the cross-validated model
pipeline = search.best_estimator_
pipeline.fit(X_train, y_train)
print('Validation accuracy:', pipeline.score(X_val, y_val))

Validation accuracy: 0.6677551020408163


## 7. Get your model's test accuracy

> (One time, at the end.)

In [16]:
print('Test accuracy:', pipeline.score(X_test, y_test))

Test accuracy: 0.6272674078408426


## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [17]:
# accuracy = total correct predictions / total predictions
correct_predictions = 85 + 36
total_predictions = 85 + 58 + 8 + 36
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy}')

Accuracy: 0.6470588235294118


### Calculate precision

In [18]:
# precision = correct value predictions of a class / total predictions for class
correct_predictions_positive = 36
total_predictions_positive = 58 + 36
precision = correct_predictions_positive / total_predictions_positive
print(f'Precision: {precision}')

Precision: 0.3829787234042553


### Calculate recall

In [19]:
# recall = correct for the class / actual for the class
actual_positive = 8 + 36
recall = correct_predictions_positive / actual_positive
print(f'Recall: {recall}')

Recall: 0.8181818181818182
